In [ ]:
# import cv2
# import face_recognition
# import os
# import numpy as np
# from qdrant_client import QdrantClient
# from qdrant_client.models import PointStruct, Distance, VectorParams
# from uuid import uuid4
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import asyncio
# from tqdm import tqdm

# # Connect to Qdrant instance via gRPC (port 6334)
# client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)

# # Define collection parameters
# collection_name = "embedding_collection1"
# vector_size = 128  # Define the size of your embedding vector
# distance_metric = Distance.COSINE  # Choose a distance metric (COSINE, EUCLIDEAN, etc.)

# # Create the collection (if it doesn't exist)
# def create():
#     client.recreate_collection(
#         collection_name=collection_name,
#         vectors_config=VectorParams(size=vector_size, distance=distance_metric)
#     )

# # Function to insert data into Qdrant
# def insert_data(name, embedding):
#     id = str(uuid4())
#     points = [PointStruct(id=id, vector=embedding, payload={"name": name})]
#     client.upsert(
#         collection_name=collection_name,
#         points=points
#     )
#     print(f"Embedding data with ID {id} successfully added to Qdrant.")

# # Function to perform a similarity search
# def compare(embedding, top_k=1):
#     threshold = 0.93
#     search_results = client.search(
#         collection_name=collection_name,
#         query_vector=embedding,
#         limit=top_k  # Number of nearest neighbors to retrieve
#     )
#     if len(search_results) == 0:
#         return "Unknown"
#     result = search_results[0]
#     if result.score >= threshold:
#         return result.payload["name"]
#     else:
#         return "Unknown"

# # Function to get embedding for a given image path
# def get_embedding(path):
#     img = cv2.imread(path)
#     convert_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     face_locations = face_recognition.face_locations(convert_rgb)
#     if len(face_locations) == 0:
#         return []
#     face_encodings = face_recognition.face_encodings(convert_rgb, known_face_locations=face_locations)
#     return face_encodings

# # Function to train data
# def train_data(path):
#     person_embeddings = {}
#     if os.path.exists(path):
#         for root, dirs, files in tqdm(os.walk(path), desc="Training Data"):
#             folder_name = os.path.basename(root)  # Get the folder name as the person name
#             person_embeddings[folder_name] = []  # Initialize an empty list for each person
#             for file in tqdm(files, desc="Processing Files"):
#                 full_path = os.path.join(root, file)
#                 embeddings = get_embedding(path=full_path)
#                 if len(embeddings) > 0:
#                     person_embeddings[folder_name].append(embeddings[0])
#         for person, embeddings_list in person_embeddings.items():
#             if embeddings_list:  # Check if there are embeddings for this person
#                 avg_embedding = np.mean(embeddings_list, axis=0)
#                 insert_data(person, avg_embedding)

# def compare_face_encoding(face_encoding, face_location):
#     result = compare(face_encoding)
#     return result, face_location

# async def predict(frame, distance_threshold=0.2):
#     fra = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     face_locations = face_recognition.face_locations(fra)
#     if len(face_locations) == 0:
#         return []
#     face_encodings = face_recognition.face_encodings(fra, known_face_locations=face_locations)
#     predictions = []
#     with ThreadPoolExecutor() as executor:
#         future_to_face = {executor.submit(compare_face_encoding, face_encoding, face_location): face_location 
#                           for face_encoding, face_location in zip(face_encodings, face_locations)}
#         for future in as_completed(future_to_face):
#             result, location = future.result()
#             predictions.append((result, location))
#     return predictions

# def show_predictions_on_frame(frame, predictions):
#     for name, (top, right, bottom, left) in predictions:
#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
#         cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_DUPLEX
#         cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)

# if __name__ == "__main__":
#     create()
#     train_data(path=r"C:\Users\sijan\Desktop\jojo\yoyo")  # Change this to your actual directory path

#     video_capture = cv2.VideoCapture(0)
#     loop = asyncio.get_event_loop()  # Get the existing event loop
#     while True:
#         ret, frame = video_capture.read()
#         if not ret:
#             break
#         predictions = loop.run_until_complete(predict(frame))  # Use run_until_complete() to run the coroutine
#         show_predictions_on_frame(frame, predictions)
#         cv2.imshow('Webcam', frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     video_capture.release()
#     cv2.destroyAllWindows()


In [ ]:
# import cv2
# import face_recognition
# import os
# import numpy as np
# from qdrant_client import QdrantClient
# from qdrant_client.models import PointStruct, Distance, VectorParams
# from uuid import uuid4
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from tqdm import tqdm
# import asyncio
# import nest_asyncio  # Fix for running asyncio in Jupyter or other environments

# nest_asyncio.apply()  # Allow asyncio to run in a running event loop

# # Connect to Qdrant instance via gRPC (port 6334)
# client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)

# # Define collection parameters
# collection_name = "embedding_collection1"
# vector_size = 128  # Define the size of your embedding vector
# distance_metric = Distance.COSINE  # Choose a distance metric (COSINE, EUCLIDEAN, etc.)

# # Create the collection (if it doesn't exist)
# def create():
#     if not client.collection_exists(collection_name=collection_name):
#         client.create_collection(
#             collection_name=collection_name,
#             vectors_config=VectorParams(size=vector_size, distance=distance_metric)
#         )
#         print(f"Collection '{collection_name}' created successfully.")
#     else:
#         print(f"Collection '{collection_name}' already exists.")

# # Function to insert data into Qdrant
# def insert_data(name, embedding):
#     id = str(uuid4())
#     points = [PointStruct(id=id, vector=embedding, payload={"name": name})]
#     client.upsert(
#         collection_name=collection_name,
#         points=points
#     )
#     print(f"Embedding data with ID {id} successfully added to Qdrant.")

# # Function to perform a similarity search
# def compare(embedding, top_k=1):
#     threshold = 0.93
#     search_results = client.search(
#         collection_name=collection_name,
#         query_vector=embedding,
#         limit=top_k
#     )
#     if len(search_results) == 0:
#         return "Unknown"
#     result = search_results[0]
#     if result.score >= threshold:
#         return result.payload["name"]
#     else:
#         return "Unknown"

# # Function to get embedding for a given image path
# def get_embedding(path):
#     img = cv2.imread(path)
#     convert_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     face_locations = face_recognition.face_locations(convert_rgb)
#     if len(face_locations) == 0:
#         return []
#     face_encodings = face_recognition.face_encodings(convert_rgb, known_face_locations=face_locations)
#     return face_encodings

# # Function to train data
# def train_data(path):
#     person_embeddings = {}
#     if os.path.exists(path):
#         for root, dirs, files in tqdm(os.walk(path), desc="Training Data"):
#             folder_name = os.path.basename(root)  # Get the folder name as the person name
#             embeddings_list = []  # Collect embeddings for averaging
#             for file in tqdm(files, desc=f"Processing Files in {folder_name}"):
#                 full_path = os.path.join(root, file)
#                 embeddings = get_embedding(path=full_path)
#                 if len(embeddings) > 0:
#                     embeddings_list.append(embeddings[0])
#             if embeddings_list:  # Calculate average embedding if there are valid embeddings
#                 avg_embedding = np.mean(embeddings_list, axis=0)
#                 insert_data(folder_name, avg_embedding)

# def compare_face_encoding(face_encoding, face_location):
#     result = compare(face_encoding)
#     return result, face_location

# async def predict(frame, distance_threshold=0.2):
#     fra = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     face_locations = face_recognition.face_locations(fra)
#     if len(face_locations) == 0:
#         return []
#     face_encodings = face_recognition.face_encodings(fra, known_face_locations=face_locations)
#     predictions = []
#     with ThreadPoolExecutor() as executor:
#         future_to_face = {executor.submit(compare_face_encoding, face_encoding, face_location): face_location 
#                           for face_encoding, face_location in zip(face_encodings, face_locations)}
#         for future in as_completed(future_to_face):
#             result, location = future.result()
#             predictions.append((result, location))
#     return predictions

# def show_predictions_on_frame(frame, predictions):
#     for name, (top, right, bottom, left) in predictions:
#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
#         cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_DUPLEX
#         cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)

# async def process_video():
#     video_capture = cv2.VideoCapture(0)
#     while True:
#         ret, frame = video_capture.read()
#         if not ret:
#             break
#         predictions = await predict(frame)
#         show_predictions_on_frame(frame, predictions)
#         cv2.imshow('Webcam', frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     video_capture.release()
#     cv2.destroyAllWindows()

# if __name__ == "__main__":
#     create()
#     train_data(path=r"C:\Users\sijan\Desktop\jojo\yoyo")  # Change this to your actual directory path
#     asyncio.run(process_video())


In [ ]:
# import os
# import cv2
# import face_recognition

# def process_images(input_dir, output_dir):
#     # Create the output directory if it doesn't exist
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for root, dirs, files in os.walk(input_dir):
#         for file in files:
#             if file.lower().endswith(('.jpg', '.jpeg', '.png')):
#                 input_path = os.path.join(root, file)

#                 # Create corresponding output folder structure
#                 relative_path = os.path.relpath(root, input_dir)
#                 output_subdir = os.path.join(output_dir, relative_path)
#                 if not os.path.exists(output_subdir):
#                     os.makedirs(output_subdir)

#                 output_path = os.path.join(output_subdir, file)

#                 # Process the image
#                 image = cv2.imread(input_path)
#                 rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#                 face_locations = face_recognition.face_locations(rgb_image, model='hog')

#                 for top, right, bottom, left in face_locations:
#                     # Expand the face box to include head to chin with ears
#                     height = bottom - top
#                     width = right - left
#                     margin = int(0.2 * height)

#                     extended_top = max(0, top - margin)
#                     extended_bottom = min(image.shape[0], bottom + margin)
#                     extended_left = max(0, left - margin)
#                     extended_right = min(image.shape[1], right + margin)

#                     face_crop = image[extended_top:extended_bottom, extended_left:extended_right]

#                     # Save the cropped face
#                     cv2.imwrite(output_path, face_crop)
#                 else:
#                     print(f"No face found in {input_path}, skipping.")

# if __name__ == "__main__":
#     input_folder = r"C:\Users\sijan\Desktop\jojo"  # Replace with the input folder path
#     output_folder = r"C:\Users\sijan\Desktop\fake_jojo"  # Replace with the output folder path
#     process_images(input_folder, output_folder)


In [ ]:
# import cv2
# import os
# import numpy as np
# from qdrant_client import QdrantClient
# from qdrant_client.models import PointStruct, Distance, VectorParams
# from uuid import uuid4
# from tqdm import tqdm
# from transformers import CLIPProcessor, CLIPModel
# import torch

# # Connect to Qdrant instance
# client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)

# # Collection parameters
# collection_name = "embedding_collection1"
# vector_size = 512  # CLIP model outputs embeddings of size 512
# distance_metric = Distance.COSINE

# # Initialize CLIP model and processor
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# # Create the collection
# def create_collection():
#     if not client.collection_exists(collection_name=collection_name):
#         client.create_collection(
#             collection_name=collection_name,
#             vectors_config=VectorParams(size=vector_size, distance=distance_metric),
#         )
#         print(f"Collection '{collection_name}' created successfully.")
#     else:
#         print(f"Collection '{collection_name}' already exists.")

# # Insert data into Qdrant
# def insert_data(name, embedding):
#     id = str(uuid4())
#     points = [PointStruct(id=id, vector=embedding.tolist(), payload={"name": name})]
#     client.upsert(collection_name=collection_name, points=points)
#     print(f"Added embedding for {name} with ID {id}.")

# # Extract embeddings for an image using CLIP
# def get_embedding(image_path):
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     inputs = processor(images=image, return_tensors="pt", padding=True)
#     with torch.no_grad():
#         embedding = model.get_image_features(**inputs).squeeze().cpu().numpy()
#     return embedding

# # Train embeddings for the dataset
# def train_data(dataset_path):
#     if not os.path.exists(dataset_path):
#         print("Dataset path does not exist.")
#         return
    
#     for root, dirs, files in tqdm(os.walk(dataset_path), desc="Processing Folders"):
#         folder_name = os.path.basename(root)
#         embeddings_list = []
        
#         for file in tqdm(files, desc=f"Processing Files in {folder_name}"):
#             full_path = os.path.join(root, file)
#             embedding = get_embedding(full_path)
#             embeddings_list.append(embedding)
        
#         if embeddings_list:
#             avg_embedding = np.mean(embeddings_list, axis=0)
#             insert_data(folder_name, avg_embedding)

# if __name__ == "__main__":
#     create_collection()
#     train_data(dataset_path=r"C:\Users\sijan\Desktop\si")


In [ ]:
# import cv2
# import asyncio
# import nest_asyncio
# from qdrant_client import QdrantClient
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from transformers import CLIPProcessor, CLIPModel
# import torch

# nest_asyncio.apply()  # Allow asyncio in environments with a running event loop

# # Connect to Qdrant instance
# client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)
# collection_name = "embedding_collection1"

# # Initialize CLIP model and processor
# model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
# processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# # Compare embeddings against the stored ones in Qdrant
# def compare(embedding, top_k=1):
#     threshold = 0.93
#     search_results = client.search(
#         collection_name=collection_name,
#         query_vector=embedding.tolist(),
#         limit=top_k
#     )
#     if not search_results:
#         return "Unknown"
#     result = search_results[0]
#     if result.score >= threshold:
#         return result.payload["name"]
#     else:
#         return "Unknown"

# # Extract embeddings for a frame using CLIP
# def get_embedding(frame):
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     inputs = processor(images=rgb_frame, return_tensors="pt", padding=True)
#     with torch.no_grad():
#         embedding = model.get_image_features(**inputs).squeeze().cpu().numpy()
#     return embedding

# # Predict face recognition for the current frame
# async def predict(frame, executor):
#     face_locations = face_recognition.face_locations(frame)
#     if not face_locations:
#         return []

#     embeddings = [get_embedding(frame) for _ in face_locations]  # Assuming the whole frame for each detected face
#     predictions = []

#     future_to_face = {
#         executor.submit(compare, embedding): face_location
#         for embedding, face_location in zip(embeddings, face_locations)
#     }
#     for future in as_completed(future_to_face):
#         result = future.result()
#         location = future_to_face[future]
#         predictions.append((result, location))

#     return predictions

# # Display predictions on the webcam feed
# def show_predictions_on_frame(frame, predictions):
#     for name, (top, right, bottom, left) in predictions:
#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
#         cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_DUPLEX
#         cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)

# # Process the webcam feed
# async def process_video():
#     video_capture = cv2.VideoCapture(0)
#     frame_skip = 5  # Process every 5th frame
#     frame_count = 0

#     with ThreadPoolExecutor(max_workers=16) as executor:
#         while True:
#             ret, frame = video_capture.read()
#             if not ret:
#                 break

#             frame_count += 1
#             if frame_count % frame_skip == 0:
#                 predictions = await predict(frame, executor)
#                 show_predictions_on_frame(frame, predictions)

#             # Display the frame
#             cv2.imshow("Webcam", frame)

#             # Exit on 'q' key press
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break

#     video_capture.release()
#     cv2.destroyAllWindows()

# if __name__ == "__main__":
#     asyncio.run(process_video())


In [ ]:
import cv2
import dlib
import os
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams
from uuid import uuid4
from tqdm import tqdm

# Connect to Qdrant instance
client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)

# Collection parameters
collection_name = "embedding_collection1"
vector_size = 128  # Vector size is model-dependent; adjust as needed
distance_metric = Distance.COSINE

# Load Dlib models
face_detector = dlib.get_frontal_face_detector()  # Face detector
shape_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Pre-trained landmarks model
face_rec_model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")  # Pre-trained face embeddings model

# Create the collection
def create_collection():
    if not client.collection_exists(collection_name=collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance=distance_metric),
        )
        print(f"Collection '{collection_name}' created successfully.")
    else:
        print(f"Collection '{collection_name}' already exists.")

# Insert data into Qdrant
def insert_data(name, embedding):
    id = str(uuid4())
    points = [PointStruct(id=id, vector=embedding.tolist(), payload={"name": name})]
    client.upsert(collection_name=collection_name, points=points)
    print(f"Added embedding for {name} with ID {id}.")

# Preprocessing: Align and resize face
def preprocess_face(image, landmarks):
    # Align the face based on eye landmarks
    left_eye = np.mean(landmarks[36:42], axis=0)  # Indices for left eye in Dlib's 68-point model
    right_eye = np.mean(landmarks[42:48], axis=0)  # Indices for right eye
    dx = right_eye[0] - left_eye[0]
    dy = right_eye[1] - left_eye[1]
    angle = np.degrees(np.arctan2(dy, dx))
    center = tuple(map(int, np.mean([left_eye, right_eye], axis=0)))  # Convert to tuple of integers
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1)
    aligned_face = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
    return aligned_face


# Extract embeddings for an image
def get_embedding(image_path):
    img = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    detections = face_detector(rgb_image, 1)  # Detect faces with Dlib

    embeddings = []
    for detection in detections:
        # Get landmarks for face
        shape = shape_predictor(rgb_image, detection)
        landmarks = np.array([[p.x, p.y] for p in shape.parts()])

        # Preprocess and align the face
        aligned_face = preprocess_face(rgb_image, landmarks)

        # Get face embedding
        embedding = face_rec_model.compute_face_descriptor(aligned_face, shape)
        embeddings.append(np.array(embedding))

    return embeddings

# Train embeddings for the dataset
def train_data(dataset_path):
    if not os.path.exists(dataset_path):
        print("Dataset path does not exist.")
        return
    
    for root, dirs, files in tqdm(os.walk(dataset_path), desc="Processing Folders"):
        folder_name = os.path.basename(root)
        embeddings_list = []
        
        for file in tqdm(files, desc=f"Processing Files in {folder_name}"):
            full_path = os.path.join(root, file)
            embeddings = get_embedding(full_path)
            if embeddings:
                embeddings_list.extend(embeddings)  # Collect all embeddings for the folder
        
        if embeddings_list:
            avg_embedding = np.mean(embeddings_list, axis=0)  # Average all embeddings for the folder
            insert_data(folder_name, avg_embedding)

if __name__ == "__main__":
    create_collection()
    train_data(dataset_path=r"C:\Users\sijan\Desktop\jojo\yoyo")


In [ ]:
import cv2
import face_recognition
import asyncio
import nest_asyncio
from qdrant_client import QdrantClient
from concurrent.futures import ThreadPoolExecutor

nest_asyncio.apply()  # Allow asyncio in environments with a running event loop

# Connect to Qdrant instance
client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)
collection_name = "embedding_collection1"

# Compare embeddings against the stored ones in Qdrant
def compare(embedding, top_k=1):
    threshold = 0.93
    search_results = client.search(
        collection_name=collection_name,
        query_vector=embedding,
        limit=top_k
    )
    if not search_results:
        return "Unknown"
    result = search_results[0]
    if result.score >= threshold:
        return result.payload["name"]
    else:
        return "Unknown"

# Process face detection and recognition
def detect_and_recognize_faces(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    if not face_locations:
        return []

    face_encodings = face_recognition.face_encodings(rgb_frame, known_face_locations=face_locations)
    results = [(compare(encoding), location) for encoding, location in zip(face_encodings, face_locations)]
    return results

# Display predictions on the frame
def show_predictions_on_frame(frame, predictions):
    for name, (top, right, bottom, left) in predictions:
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)

# Asynchronous video processing
async def process_video():
    video_capture = cv2.VideoCapture(0)
    frame_skip = 1  # Process every nth frame
    frame_count = 0

    with ThreadPoolExecutor(max_workers=8) as executor:  # Adjust max_workers based on system capability
        loop = asyncio.get_event_loop()

        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            frame_count += 1
            if frame_count % frame_skip == 0:
                # Run face detection and recognition in a thread
                predictions = await loop.run_in_executor(executor, detect_and_recognize_faces, frame)
                show_predictions_on_frame(frame, predictions)

            # Display the frame
            cv2.imshow("Webcam", frame)

            # Exit on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    video_capture.release()
    cv2.destroyAllWindows()

# Main entry point
if __name__ == "__main__":
    try:
        asyncio.run(process_video())
    except KeyboardInterrupt:
        print("\nExiting...")


# Use Deepface

In [ ]:
# import os
# import cv2
# import numpy as np
# from deepface import DeepFace
# from qdrant_client import QdrantClient
# from qdrant_client.models import PointStruct, Distance, VectorParams
# from uuid import uuid4
# from tqdm import tqdm

# # Connect to Qdrant instance
# client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)

# # Collection parameters
# collection_name = "embedding_collection1"
# vector_size = 4096  # Reduced to 128-dimensional embedding
# distance_metric = Distance.COSINE

# # Create the collection
# def create_collection():
#     if not client.collection_exists(collection_name=collection_name):
#         client.create_collection(
#             collection_name=collection_name,
#             vectors_config=VectorParams(size=vector_size, distance=distance_metric),
#         )
#         print(f"Collection '{collection_name}' created successfully.")
#     else:
#         print(f"Collection '{collection_name}' already exists.")

# # Insert data into Qdrant
# def insert_data(name, embedding):
#     id = str(uuid4())
#     # Ensure embedding is exactly 128 dimensions
#     # embedding = embedding[:128]
#     points = [PointStruct(id=id, vector=embedding, payload={"name": name})]
#     client.upsert(collection_name=collection_name, points=points)
#     print(f"Added embedding for {name} with ID {id}.")

# # Extract embeddings for an image using DeepFace
# def get_embedding(image_path):
#     try:
#         # Use DeepFace to extract face embedding
#         # Reduce embedding to 128 dimensions
#         embedding_objs = DeepFace.represent(
#             img_path=image_path, 
#             model_name='VGG-Face',
#             enforce_detection=False,
#             detector_backend='opencv',  # More robust face detection
#             align=True,  # Ensure face alignment
#             normalization='base'  # Normalize the embedding
#         )
#         print(len(embedding_objs[0]['embedding']))
#         # If multiple faces are detected, take the first embedding
#         if embedding_objs:
#             # Truncate or pad the embedding to exactly 128 dimensions
#             full_embedding = embedding_objs[0]['embedding']
#             return full_embedding
#         return None
#     except Exception as e:
#         print(f"Error processing {image_path}: {e}")
#         return None

# # Train embeddings for the dataset
# def train_data(dataset_path):
#     if not os.path.exists(dataset_path):
#         print("Dataset path does not exist.")
#         return
    
#     for root, dirs, files in tqdm(os.walk(dataset_path), desc="Processing Folders"):
#         folder_name = os.path.basename(root)
#         embeddings_list = []
        
#         for file in tqdm(files, desc=f"Processing Files in {folder_name}"):
#             # Check for image files
#             if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
#                 full_path = os.path.join(root, file)
#                 embedding = get_embedding(full_path)
#                 if embedding is not None:
#                     embeddings_list.append(embedding)
        
#         # Compute average embedding if multiple faces found
#         if embeddings_list:
#             # avg_embedding = np.mean(embeddings_list, axis=0).tolist()
#             # Ensure exactly 128 dimensions
#             # avg_embedding = avg_embedding[:128] + [0] * max(0, 128 - len(avg_embedding))
#             for embedding in embeddings_list:
#                 insert_data(folder_name, embedding)

# if __name__ == "__main__":
#     # Ensure you have the necessary DeepFace models downloaded
#     # You might need to run: 
#     # DeepFace.download_model('VGG-Face')
    
#     create_collection()
#     train_data(dataset_path=r"C:\Users\sijan\Desktop\jojo\yoyo")

# # Installation notes:
# # pip install deepface qdrant-client opencv-python tqdm numpy
# # 
# # Additional configuration:
# # - Uses VGG-Face model
# # - Reduces embedding to 128 dimensions
# # - Handles variable-length embeddings
# # - Provides robust face detection and alignment

In [ ]:
# import cv2
# import numpy as np
# from deepface import DeepFace
# from qdrant_client import QdrantClient

# # Connect to Qdrant instance
# client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)
# collection_name = "embedding_collection1"

# # Compare embeddings against the stored ones in Qdrant
# def compare_embedding(embedding, threshold=0.93):
#     try:
#         # Search in Qdrant collection
#         search_results = client.search(
#             collection_name=collection_name,
#             query_vector=embedding,
#             limit=1  # Top 1 result
#         )
        
#         # If no results found
#         if not search_results:
#             return "Unknown"
        
#         # Check if similarity meets threshold
#         result = search_results[0]
#         print(result.payload["name"])
#         print(result.score)
#         if result.score >= threshold:
#             return result.payload["name"]
#         else:
#             return "Unknown"
    
#     except Exception as e:
#         print(f"Comparison error: {e}")
#         return "Unknown"

# # Predict faces in a frame
# def predict_faces(frame):
#     predictions = []
    
#     try:
#         # Detect faces
#         faces = DeepFace.extract_faces(
#             img_path=frame, 
#             enforce_detection=False,
#             detector_backend='opencv'
#         )
        
#         # Process each detected face
#         for face in faces:
#             # Get face region
#             region = face['facial_area']
#             left, top = region['x'], region['y']
#             right, bottom = left + region['w'], top + region['h']
            
#             # Extract face from original frame
#             face_img = frame[top:bottom, left:right]
            
#             try:
#                 # Get face embedding
#                 embedding_obj = DeepFace.represent(
#                     img_path=face_img, 
#                     model_name='VGG-Face',
#                     enforce_detection=False
#                 )
                
#                 # Truncate to 128 dimensions if needed
#                 if embedding_obj:
#                     embedding = embedding_obj[0]['embedding']
                    
#                     # Compare embedding
#                     name = compare_embedding(embedding)
                    
#                     # Add prediction
#                     predictions.append({
#                         'name': name,
#                         'location': (top, right, bottom, left)
#                     })
            
#             except Exception as e:
#                 print(f"Embedding extraction error: {e}")
        
#     except Exception as e:
#         print(f"Face detection error: {e}")
    
#     return predictions

# # Visualize predictions on frame
# def draw_predictions(frame, predictions):
#     for pred in predictions:
#         name = pred['name']
#         (top, right, bottom, left) = pred['location']
        
#         # Draw rectangle around the face
#         cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        
#         # Put name label
#         cv2.rectangle(frame, (left, bottom), (right, bottom + 35), (0, 255, 0), cv2.FILLED)
#         font = cv2.FONT_HERSHEY_SIMPLEX
#         cv2.putText(frame, name, (left + 6, bottom + 30), 
#                     font, 0.8, (255, 255, 255), 2)
    
#     return frame

# # Real-time face recognition
# def real_time_recognition():
#     # Open webcam
#     video_capture = cv2.VideoCapture(0)
    
#     while True:
#         # Capture frame-by-frame
#         ret, frame = video_capture.read()
        
#         if not ret:
#             break
        
#         # Predict faces
#         predictions = predict_faces(frame)
        
#         # Draw predictions
#         annotated_frame = draw_predictions(frame, predictions)
        
#         # Display the resulting frame
#         cv2.imshow('Face Recognition', annotated_frame)
        
#         # Exit on 'q' press
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
    
#     # Release resources
#     video_capture.release()
#     cv2.destroyAllWindows()

# # Main execution
# if __name__ == "__main__":
#     real_time_recognition()

# # Required dependencies:
# # pip install deepface qdrant-client opencv-python numpy
# #
# # Notes:
# # - Connects to previously created Qdrant collection
# # - Uses DeepFace for face detection and embedding
# # - Performs real-time face recognition
# # - Handles multiple face detection in a single frame

In [1]:
import cv2
import face_recognition
import os
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams
from uuid import uuid4
from tqdm import tqdm

# Connect to Qdrant instance
client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)

# Collection parameters
collection_name = "embedding_collection1"
vector_size = 128
distance_metric = Distance.COSINE

# Create the collection
def create_collection():
    if not client.collection_exists(collection_name=collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance=distance_metric),
        )
        print(f"Collection '{collection_name}' created successfully.")
    else:
        print(f"Collection '{collection_name}' already exists.")

# Insert data into Qdrant
def insert_data(name, embedding):
    id = str(uuid4())
    points = [PointStruct(id=id, vector=embedding, payload={"name": name})]
    client.upsert(collection_name=collection_name, points=points)
    print(f"Added embedding for {name} with ID {id}.")

# Extract embeddings for an image
def get_embedding(image_path):
    img = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_image)
    if not face_locations:
        return []
    face_encodings = face_recognition.face_encodings(rgb_image, known_face_locations=face_locations)
    return face_encodings

# Train embeddings for the dataset
def train_data(dataset_path):
    if not os.path.exists(dataset_path):
        print("Dataset path does not exist.")
        return
    
    for root, dirs, files in tqdm(os.walk(dataset_path), desc="Processing Folders"):
        folder_name = os.path.basename(root)
        embeddings_list = []
        
        for file in tqdm(files, desc=f"Processing Files in {folder_name}"):
            full_path = os.path.join(root, file)
            embeddings = get_embedding(full_path)
            if embeddings:
                embeddings_list.append(embeddings[0])
        
        if embeddings_list:
            avg_embedding = np.mean(embeddings_list, axis=0)
            insert_data(folder_name, avg_embedding)

if __name__ == "__main__":
    create_collection()
    train_data(dataset_path=r"C:\Users\sijan\Desktop\jojo\yoyo")


Collection 'embedding_collection1' created successfully.


Processing Files in yoyo: 0it [00:00, ?it/s]
Processing Files in safal: 100%|██████████| 11/11 [01:43<00:00,  9.42s/it]
Processing Folders: 2it [01:43, 51.80s/it]

Added embedding for safal with ID 4d968a03-40e7-4222-b0aa-fa24b78731c1.


Processing Files in sijan: 100%|██████████| 11/11 [01:29<00:00,  8.13s/it]
Processing Folders: 3it [03:13, 64.36s/it]

Added embedding for sijan with ID cdbbe339-bcd4-4e2c-a3d2-ac8cca68f238.


In [3]:
import cv2
import face_recognition
import asyncio
import nest_asyncio
from qdrant_client import QdrantClient
from concurrent.futures import ThreadPoolExecutor, as_completed

nest_asyncio.apply()  # Allow asyncio in environments with a running event loop

# Connect to Qdrant instance
client = QdrantClient(host="localhost", port=6334, prefer_grpc=True)
collection_name = "embedding_collection1"

# Compare embeddings against the stored ones in Qdrant
def compare(embedding, top_k=1):
    threshold = 0.93
    search_results = client.search(
        collection_name=collection_name,
        query_vector=embedding,
        limit=top_k
    )
    if not search_results:
        return "Unknown"
    result = search_results[0]
    if result.score >= threshold:
        return result.payload["name"]
    else:
        return "Unknown"

# Predict face recognition for the current frame
async def predict(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    if not face_locations:
        return []
    
    face_encodings = face_recognition.face_encodings(rgb_frame, known_face_locations=face_locations)
    predictions = []

    with ThreadPoolExecutor() as executor:
        future_to_face = {
            executor.submit(compare, face_encoding): face_location
            for face_encoding, face_location in zip(face_encodings, face_locations)
        }
        for future in as_completed(future_to_face):
            result = future.result()
            location = future_to_face[future]
            predictions.append((result, location))
    
    return predictions

# Display predictions on the webcam feed
def show_predictions_on_frame(frame, predictions):
    for name, (top, right, bottom, left) in predictions:
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)

# Process the webcam feed
async def process_video():
    video_capture = cv2.VideoCapture(0)
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        predictions = await predict(frame)
        show_predictions_on_frame(frame, predictions)
        cv2.imshow("Webcam", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    asyncio.run(process_video())

In [1]:
import torch
print(torch.cuda.is_available())

True
